In [1]:
#Import needed packages
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import math 

In [2]:
#Read in CSV and drop the ID column
test = pd.read_csv('test.csv')
test.drop('Id', axis = 1, inplace = True)

In [3]:
#Creating new features
#Season: combining the year and season to know which season and year the house was sold in 
#newHouse: to see if the house was sold the same year as it was purchased
#remodeled: to see if the house was remodeled before it was sold the same year as it was purchased
#randInit: generating a column of random numbers to filter on later

test['remodeled'] = [1 if (test['YearBuilt'].iloc[i] == (test['YearRemodAdd'].iloc[i])) else 0 for i in range(0,1459)]
test['newHouse'] = [1 if (test['YearBuilt'].iloc[i] == (test['YrSold'].iloc[i])) else 0 for i in range(0,1459)]
arr = []
for i in range(0,1459):
    if test['MoSold'].iloc[i] in [12, 1, 2]: 
        arr.append('Spring' + "-" + str(test['YrSold'].iloc[i]))
    elif test['MoSold'].iloc[i] in [3, 4, 5]:
        arr.append('Summer' + "-" + str(test['YrSold'].iloc[i]))
    elif test['MoSold'].iloc[i] in [3, 4, 5]: 
        arr.append('Fall' + "-" + str(test['YrSold'].iloc[i]))
    else:
        arr.append('Winter' + "-" + str(test['YrSold'].iloc[i]))
test['season'] = arr
randInit = np.random.choice(np.arange(0,10000), size=1459)
randCol = pd.DataFrame(randInit)
randCol.columns = ['randInit']
test['randInit'] = randCol['randInit']

In [4]:
#Drop MiscFeature and PoolQC as there were too many nulls
test = test.drop(['MiscFeature', 'PoolQC'], axis = 1)

In [5]:
#Specific nulls to the test dataset 
test.loc[test.BsmtFullBath.isnull(), 'BsmtFullBath'] = int(0)
test.BsmtFullBath = test.BsmtFullBath.astype(int)
test.loc[test.BsmtHalfBath.isnull(), 'BsmtHalfBath'] = int(0)
test.BsmtHalfBath = test.BsmtHalfBath.astype(int)
test.loc[test.BsmtFinSF1.isnull(), 'BsmtFinSF1'] = int(0)
test.BsmtFinSF1 = test.BsmtFinSF1.astype(int)
test.loc[test.BsmtFinSF2.isnull(), 'BsmtFinSF2'] = int(0)
test.BsmtFinSF2 = test.BsmtFinSF2.astype(int)
test.loc[test.BsmtUnfSF.isnull(), 'BsmtUnfSF'] = int(0)
test.BsmtUnfSF = test.BsmtUnfSF.astype(int)
test.loc[test.TotalBsmtSF.isnull(), 'TotalBsmtSF'] = int(0)
test.TotalBsmtSF = test.TotalBsmtSF.astype(int)
test.loc[test.SaleType.isnull(), 'SaleType'] = 'WD'
test.loc[test.Exterior1st.isnull(), 'Exterior1st'] = "BrkFace"
test.loc[test.Exterior2nd.isnull(), 'Exterior2nd'] = "BrkFace"
test.loc[test.KitchenQual.isnull(), 'KitchenQual'] = "TA"
test.loc[test.GarageArea.isnull(), "GarageArea"] = int(0)
test.GarageArea = test.GarageArea.astype(int)
test.loc[test.GarageCars.isnull(), "GarageCars"] = int(0)
test.GarageCars = test.GarageCars.astype(int)
test.loc[test.Utilities.isnull(), 'Utilities'] = 'AllPub'
test.loc[test.Functional.isnull(), 'Functional'] = 'Mod'
test.loc[test.MSZoning.isnull(), 'MSZoning'] = 'C (all)'

In [6]:
#Masonry data was missing. Replace with 0 and None.
test['MasVnrType'].fillna(value = 'None', inplace = True)
test['MasVnrArea'].replace(np.nan, 0, inplace = True)

#Changing some of the columns to ordinal values
mapping = {'Gtl': 0, 'Mod': 1, 'Sev': 2}
test.replace({'LandSlope' : mapping}, inplace = True)
mapping = {'Reg': 0, 'IR1': 1, 'IR2': 2, 'IR3': 3}
test.replace({'LotShape' : mapping}, inplace = True)
mapping = {'Grvl': 0, 'Pave': 1}
test.replace({'Street' : mapping}, inplace = True)

#Alley and Basements have the string NA, which is not missing. Alley and other variables have NA on purpose.
#It needs to be convert to string. All basement variables were consistent.
test.loc[test.Alley.isnull(), 'Alley'] = 'NoAlley'
test.loc[test.BsmtCond.isnull(), 'BsmtCond'] = 'NoBasement'
test.loc[test.BsmtExposure.isnull(), 'BsmtExposure'] = 'NoBasement'
test.loc[test.BsmtFinType1.isnull(), 'BsmtFinType1'] = 'NoBasement'
test.loc[test.BsmtFinType2.isnull(), 'BsmtFinType2'] = 'NoBasement'
test.loc[test.BsmtQual.isnull(), 'BsmtQual'] = 'NoBasement'
test['Fence'].replace(np.nan, 'NoFence', inplace=True)
test['FireplaceQu'].replace(np.nan, 'NoFireplace', inplace=True)
test['GarageCond'].replace(np.nan, 'NoGarage', inplace=True)
test['GarageFinish'].replace(np.nan, 'NoGarage', inplace=True)
test['GarageQual'].replace(np.nan, 'NoGarage', inplace=True)
test['GarageType'].replace(np.nan, 'NoGarage', inplace=True)

#Central Air changed to 1 or 0.
centralair = {'N': 0,'Y': 1}
test.CentralAir = [centralair[i] for i in test.CentralAir]

#Further missing values
test['randInit'].replace(np.nan, '2138', inplace=True)
test['Electrical'].replace(np.nan, 'SBrkr', inplace=True)

In [7]:
#Changing the nulls to imputed values
#Using Lot Area to fill in values for Lot Frontage.
#Adding together all SF columns to make a total SF column
test['GarageYrBlt'].replace(np.nan, test['GarageYrBlt'].mean(), inplace = True)
test['areaBin'] = pd.cut(test.LotArea, bins = 10).astype(str)
X = test.groupby('areaBin')[['LotFrontage']].median()
X.reset_index(inplace=True)
X['areaBin'] = X['areaBin'].astype(str)
test['LotFrontage'] = [test.LotFrontage.iloc[i] if not np.isnan(test.LotFrontage.iloc[i])
          else X[X.areaBin == test.iloc[i]['areaBin']]['LotFrontage'].iloc[0] for i in range(len(test))]

test['TotalSF'] = (test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF'] + test['WoodDeckSF'] + 
                test['OpenPorchSF'] + test['EnclosedPorch'] + test['3SsnPorch'] + test['ScreenPorch'] +
                test['GarageArea'])
test["LotFrontage"] = test["LotFrontage"].replace(np.nan, 106)

In [8]:
#train.to_csv('no_dums.csv', index = False)
test.to_csv('test_updated.csv', index = False)